In [1]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7fbb0fe9a490>)

In [2]:
%load_ext line_profiler

In [8]:
import utils
import utils_opt
import sknn_opt
import sknn 
import pandas as pd
import numpy as np

In [23]:
FOLDER_TRAIN = 'data/train/short_ver/'
FOLDER_TEST = 'data/test/'
OUTPUT_FOLDER = 'output/'

In [24]:
%time train, test = utils.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.26732897758483887s
 -- filtered in: 0.373823881149292s
 -- actions:  664712
 -- items:  170089
 -- lists:  10000
CPU times: user 340 ms, sys: 109 ms, total: 448 ms
Wall time: 430 ms


In [25]:
%time train, test = utils_opt.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.24930095672607422s
 -- filtered in: 0.35671496391296387s
(' -- actions: ', 664712)
(' -- items: ', 170089)
(' -- lists: ', 10000)
CPU times: user 341 ms, sys: 105 ms, total: 446 ms
Wall time: 416 ms


In [38]:
train, test = utils.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.2947571277618408s
 -- filtered in: 0.395413875579834s
 -- actions:  664712
 -- items:  170089
 -- lists:  10000


In [39]:
rec = sknn.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [28]:
rec.train(train, test)

In [40]:
%time rec.train(train, test)

CPU times: user 452 ms, sys: 62 ms, total: 514 ms
Wall time: 514 ms


In [29]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

In [30]:
tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

In [31]:
rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 

,track_id,confidence
0,1921,2.228768
1,802,1.623090
2,1244,1.087304
3,2398,1.041252
4,801,1.032605
...,...,...
495,6744,0.210092
496,7731,0.210040
497,8402,0.209968
498,65,0.209734


### Attempted optimized approach

In [ ]:
# %lprun -f rec.train rec.train(train, test)

In [32]:
train, test = utils_opt.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.2729170322418213s
 -- filtered in: 0.37807393074035645s
(' -- actions: ', 664712)
(' -- items: ', 170089)
(' -- lists: ', 10000)


In [33]:
rec = sknn_opt.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [34]:
%time rec.train(train, test)

CPU times: user 346 ms, sys: 57.1 ms, total: 403 ms
Wall time: 402 ms


In [35]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

In [36]:
tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

In [37]:
rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 

,track_id,confidence
0,1921,2.228768
1,802,1.623090
2,1244,1.087304
3,2398,1.041252
4,801,1.032605
...,...,...
495,6744,0.210092
496,7731,0.210040
497,8402,0.209968
498,65,0.209734
